In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import seaborn as sns
import os
from glob import glob

# import ecCodes

In [11]:
# import cfgrib
# !pip install ecCodes
# !conda install eccodes
# !conda install -c conda-forge eccodes

In [12]:
# read table with params: 
outdata = '/Users/varyabazilova/Desktop/alluvial_fans/dem_all/all_points_so_far/10_datawrangl_formodel!!/out/'

dfclim = pd.read_csv(outdata + '2aug_morph_temperature.csv')

In [13]:
dfclim.head()

,Unnamed: 0,DN,target,Name,x_centroid,y_centroid,area_m,perimeter,glaciersum,mean_elevation,...,Rc,Cc,mean_annual_T,mean_jan_T,mean_july_T,mean_monsoon_T,mean_no_monsoon_T,count_zeros,belowzero_frac,avgtemp_belowzero
0,0,2,1,DF,6533932.497,-500205.440,323908.568,3584.675,0.000,4738.964789,...,0.316762,1.789410,-4.698425,-13.451904,4.031158,0.353302,-9.77121,6.6875,0.66738,1
1,1,3,1,DF,6534113.836,-500799.045,182483.700,2922.889,0.000,4568.158333,...,0.268417,1.943889,-4.698425,-13.451904,4.031158,0.353302,-9.77121,6.6875,0.66738,1
2,3,4,1,DF,6533894.325,-503821.445,205294.163,2702.293,0.000,4182.414815,...,0.353282,1.694398,-4.698425,-13.451904,4.031158,0.353302,-9.77121,6.6875,0.66738,1
3,7,5,0,"Mixed - new looks FF, vegetated DF",6531952.768,-502250.739,8413968.867,19907.851,2763.123,4867.525384,...,0.266785,1.949823,-4.698425,-13.451904,4.031158,0.353302,-9.77121,6.6875,0.66738,1
4,8,7,0,FF - lower lef DF remnants?,6535837.722,-503989.910,359644.959,3750.121,0.000,4346.589852,...,0.321361,1.776559,-4.698425,-13.451904,4.031158,0.353302,-9.77121,6.6875,0.66738,1


1. Monsoon precipitation (sum of June-sept) 
2. Mean precipitation over year - daily
3. Annual sum 
4. Fraction of precipitation in monsoon 
5. P95% daily - sum in a day and then 95% 
6. Average number of wet days per year (frequency) 
7. Rain intensity on wet days 
8. Snow/no snow (fraction?) during wet days - if temp that day was below 0 precip_daymax_1990.nc
9. N-day sum (?) — rainy days (10 to 30) 
10. Monthly average -> cv (std/average) 

In [14]:
testfile_p = xr.open_dataset('/Users/varyabazilova/Desktop/era5land/dayly_precip/precip_daymax_1990.nc')

testfile_t = xr.open_dataset('/Users/varyabazilova/Desktop/era5land/daymean_temp/daymean_temp_1990.nc')

In [15]:
testfile_t

<xarray.Dataset>
Dimensions:    (time: 365, bnds: 2, longitude: 691, latitude: 491)
Coordinates:
  * time       (time) datetime64[ns] 1990-01-01T11:00:00 ... 1990-12-31T11:00:00
  * longitude  (longitude) float32 55.0 55.1 55.2 55.3 ... 123.8 123.9 124.0
  * latitude   (latitude) float32 55.0 54.9 54.8 54.7 54.6 ... 6.3 6.2 6.1 6.0
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] ...
    t2m        (time, latitude, longitude) float32 ...
Attributes:
    CDI:          Climate Data Interface version 2.0.5 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    history:      Tue Jul 26 15:57:14 2022: cdo daymean temp_2m_year1990.nc d...
    frequency:    day
    CDO:          Climate Data Operators version 2.0.5 (https://mpimet.mpg.de...

In [16]:
testfile_p

<xarray.Dataset>
Dimensions:    (time: 365, bnds: 2, longitude: 691, latitude: 491)
Coordinates:
  * time       (time) datetime64[ns] 1990-01-01T11:00:00 ... 1990-12-31T11:00:00
  * longitude  (longitude) float32 55.0 55.1 55.2 55.3 ... 123.8 123.9 124.0
  * latitude   (latitude) float32 55.0 54.9 54.8 54.7 54.6 ... 6.3 6.2 6.1 6.0
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] ...
    tp         (time, latitude, longitude) float32 ...
Attributes:
    CDI:          Climate Data Interface version 2.0.5 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    history:      Mon Aug 01 12:05:05 2022: cdo daymax total_precip_year1990....
    frequency:    day
    CDO:          Climate Data Operators version 2.0.5 (https://mpimet.mpg.de...

## just precipitations (no temps) 


### mean annual daily precipitation

In [23]:
%%time
# calculate mean annual temperature from daily data for each year: 

path = '/Users/varyabazilova/Desktop/era5land/dayly_precip'

meanannual = xr.Dataset()

for f in os.listdir(path):
    # read every file: 
    ds = xr.open_dataset(os.path.join(path, f), decode_coords="all", engine ='netcdf4')
    # drop useless dimention
    ds = ds.drop(labels='time_bnds')
    # group by -> resample to annual 
    dsmean = ds.groupby('time.year').mean('time')
    # print(dsmean)
    # merge 
    meanannual = xr.merge([meanannual, dsmean])

# print(meanannual)

meanannual_mean = meanannual.mean(dim='year')

OSError: [Errno -51] NetCDF: Unknown file format: b'/Users/varyabazilova/Desktop/era5land/dayly_precip/.DS_Store'

In [ ]:
# pip install netCDF4

In [ ]:
# pip install  netcdf4 h5netcdf scipy pydap zarr fsspec cftime rasterio cfgrib pooch


In [ ]:
# pip install netcdf4 h5netcdf scipy pydap rasterio zarr

In [ ]:
# pip install cfgrib

In [ ]:
# !pip install eccodes

In [ ]:
!conda install xarray dask netCDF4 bottleneck

Solving environment: \ 

In [19]:
# !pip install netCDF4 